**Ejercicio 7: Bases de Datos Vectoriales**

**Nombre:** Aaròn Yumancela

**Objetivo de la práctica**


Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

**Parte 0: Cargar el corpus**

In [ ]:
from google.colab import files
import zipfile, os, glob
import pandas as pd

uploaded = files.upload()
zip_name = list(uploaded.keys())[0]
print("ZIP subido:", zip_name)

extract_dir = "dataset_zip"
os.makedirs(extract_dir, exist_ok=True)

# extraemos el dataset
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall(extract_dir)
# verificamos archivos extraìdos
print("Contenido extraído en:", extract_dir)
print("Ejemplos de archivos:", glob.glob(extract_dir + "/**/*", recursive=True)[:20])

# buscamos el CSV dentro del zip
csv_candidates = glob.glob(extract_dir + "/**/*.csv", recursive=True)
print("CSVs encontrados:", csv_candidates[:10])

target_name = "wikipedia_text_corpus.csv"
csv_path = None
for p in csv_candidates:
    if os.path.basename(p) == target_name:
        csv_path = p
        break
if csv_path is None:
    csv_path = csv_candidates[0]



Saving dataset_.zip to dataset_.zip
ZIP subido: dataset_.zip
Contenido extraído en: dataset_zip
Ejemplos de archivos: ['dataset_zip/wikipedia_text_corpus.csv']
CSVs encontrados: ['dataset_zip/wikipedia_text_corpus.csv']


In [ ]:
print("Usando CSV:", csv_path)

df = pd.read_csv(csv_path)
df.head()


Usando CSV: dataset_zip/wikipedia_text_corpus.csv


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


**Parte 1: Generación de Embeddings**

In [ ]:
import re
import numpy as np
import pandas as pd

# limpiamos nulos
df = df.dropna(subset=["text"]).reset_index(drop=True)

# aplicamos normalización básica
def normalize_text(s: str) -> str:
  # dentro de estaa funciòn quitamos espacios repetidos
    s = str(s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].map(normalize_text)
df.head()


,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    chunks = []
    start = 0
    n = len(text)
    # dividomos texto en partes
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end == n:
            break
        # mantenemos solapamiento fijo
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "id": f"{i}_{j}",
            "doc_id": int(i),
            "chunk_id": int(j),
            "text": ch
        })

chunks_df = pd.DataFrame(records)
print("Chunks:", len(chunks_df))
chunks_df.head()


Chunks: 79104


,id,doc_id,chunk_id,text
0,0_0,0,0,Anovo Anovo (formerly A Novo) is a computer se...
1,1_0,1,0,Battery indicator A battery indicator (also kn...
2,1_1,1,1,ad battery when in reality it indicates a prob...
3,1_2,1,2,s that an internal standby battery needs repla...
4,1_3,1,3,increase; in many cases the EMF remains more o...


In [ ]:
!pip -q install sentence-transformers

from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"
model = SentenceTransformer(MODEL_NAME)

# E5 recomienda prefijos: "passage:" y "query:"
passages = ["passage: " + t for t in chunks_df["text"].tolist()]
# generamos embeddings normalizados
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

print("Embeddings:", embeddings.shape, embeddings.dtype)

def embed_query(query: str) -> np.ndarray:
  # prefijo para la consulta
    q = "query: " + query
    vec = model.encode([q], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    return vec

query_text = "Battery measuring"
query_embedding = embed_query(query_text)
print("Query vec:", query_embedding.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

Embeddings: (79104, 768) float32
Query vec: (1, 768)


In [ ]:
texts = chunks_df["text"].tolist()
metadatas = [{
    "doc_id": int(r["doc_id"]),
    "chunk_id": int(r["chunk_id"])
} for _, r in chunks_df.iterrows()]


**Parte 2: FAISS**

In [19]:
!pip -q install faiss-cpu

import faiss
D = embeddings.shape[1]
# creamos índice FAISS
index = faiss.IndexFlatL2(D)
# insertamos los vectore
index.add(embeddings)

k = 10
# buscamos vecinos cercanos
distances, idxs = index.search(query_embedding, k)

print("Top resultados (FAISS L2):")
for rank in range(k):
    i = int(idxs[0, rank])
    print(rank+1, "dist=", float(distances[0, rank]), "| id=", chunks_df.loc[i, "id"])
    print("text:", chunks_df.loc[i, "text"][:200], "...\n")


Top resultados (FAISS L2):
1 dist= 0.2593029737472534 | id= 1391_0
text: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it ...

2 dist= 0.27639928460121155 | id= 1_0
text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It ...

3 dist= 0.3197968602180481 | id= 1391_1
text: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after having applie ...

4 dist= 0.32173341512680054 | id= 5067_1
text: ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. T

**Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)**

In [20]:
!pip -q install qdrant-client


In [21]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct


with open("ri_tarea_api_key.txt", "r") as f:
    QDRANT_API_KEY = f.read().strip()

QDRANT_URL = "https://50ad3f7a-2ea9-4e20-a3cc-ffc469f7a492.us-east4-0.gcp.cloud.qdrant.io"

from qdrant_client import QdrantClient
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)



In [22]:
D = embeddings.shape[1]
COLLECTION = "wiki_chunks"

client.recreate_collection(
    collection_name=COLLECTION,
    vectors_config=VectorParams(
        size=D,
        distance=Distance.COSINE
    )
)


/tmp/ipython-input-1206355961.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [23]:
batch_size = 1000
points = []
# subimos datos por lotes
for i in range(len(texts)):
    points.append(
        PointStruct(
            id=i,
            vector=embeddings[i].tolist(),
            payload={
                "text": texts[i],
                "doc_id": metadatas[i]["doc_id"],
                "chunk_id": metadatas[i]["chunk_id"]
            }
        )
    )
    if len(points) == batch_size:
        client.upsert(collection_name=COLLECTION, points=points)
        points = []

if points:
    client.upsert(collection_name=COLLECTION, points=points)

print("Insertado en Qdrant:", len(texts))


Insertado en Qdrant: 79104


In [24]:
def qdrant_search(query_embedding, k=5):
  # consulta vectorial en Qdrant
    res = client.query_points(
        collection_name=COLLECTION,
        query=query_embedding[0].tolist(),
        limit=k,
        with_payload=True
    )

    out = []
    for r in res.points:
        payload = r.payload or {}
        out.append((
            r.id,
            r.score,
            payload.get("text", ""),
            {
                "doc_id": payload.get("doc_id"),
                "chunk_id": payload.get("chunk_id")
            }
        ))
    return out


In [25]:
results = qdrant_search(query_embedding, k=5)

for r in results:
    print("id:", r[0], "score:", r[1], "meta:", r[3])
    print("text:", r[2][:200], "...\n")


id: 10176 score: 0.8703487 meta: {'doc_id': 1391, 'chunk_id': 0}
text: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it ...

id: 1 score: 0.86180043 meta: {'doc_id': 1, 'chunk_id': 0}
text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It ...

id: 10177 score: 0.8401017 meta: {'doc_id': 1391, 'chunk_id': 1}
text: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after having applie ...

id: 37406 score: 0.8391334 meta: {'doc_id': 5067, 'chunk_id': 1}
text: ils. One was connected via a series resistor to the battery supply. The second was connected to the same

**¿La métrica usada fue cosine o L2? ¿Por qué?**

Cosine, porque los embeddings se generaron con normalize_embeddings=True, y para texto suele funcionar mejor comparar dirección (semántica) que magnitud.

**¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?**

En Qdrant es directo usando filtros sobre el payload (metadata). En FAISS, el índice solo devuelve IDs, así que el filtrado se hace manual después de la búsqueda.

**¿Qué pasa con el tiempo de respuesta cuando aumentas k?**

Aumenta el tiempo (latencia), porque el motor debe recuperar y ordenar más candidatos y además devolver más payloads. En general el costo crece con k (no siempre lineal, pero sí aumenta).



**Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)**

In [26]:
!pip -q install pymilvus


In [27]:
with open("zilliz_api_key.txt", "r") as f:
    ZILLIZ_API_KEY = f.read().strip()

ZILLIZ_URI = "https://in03-fb7b6ab34ce9ae5.serverless.aws-eu-central-1.cloud.zilliz.com"


In [28]:
from pymilvus import connections
# conectamos a Milvus cloud
connections.connect(
    alias="default",
    uri=ZILLIZ_URI,
    token=ZILLIZ_API_KEY
)


In [29]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility

COL = "wiki_milvus"
D = embeddings.shape[1]
# reiniciamos la colección
if utility.has_collection(COL):
    utility.drop_collection(COL)

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=D),
    FieldSchema(name="doc_id", dtype=DataType.INT64),
    FieldSchema(name="chunk_id", dtype=DataType.INT64),
]

schema = CollectionSchema(fields, description="Wikipedia chunks embeddings")
collection = Collection(name=COL, schema=schema)


In [30]:
batch_size = 1000

ids = list(range(len(texts)))
emb_list = embeddings.tolist()
doc_ids = [m["doc_id"] for m in metadatas]
chunk_ids = [m["chunk_id"] for m in metadatas]
# insertamos vectores y metadata
for i in range(0, len(ids), batch_size):
    collection.insert([
        ids[i:i+batch_size],
        emb_list[i:i+batch_size],
        doc_ids[i:i+batch_size],
        chunk_ids[i:i+batch_size],
    ])

collection.flush()


In [31]:
index_params = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 16, "efConstruction": 200}
}
# creamos índice HNSW
collection.create_index(
    field_name="embedding",
    index_params=index_params
)

collection.load()


In [32]:
import time

def milvus_search(query_embedding, k=5, ef=64):
    search_params = {
        "metric_type": "COSINE",
        "params": {"ef": ef}
    }
    # medimos latencia de consulta
    t0 = time.time()
    res = collection.search(
        data=query_embedding.tolist(),
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["doc_id", "chunk_id"]
    )
    t1 = time.time()

    out = []
    for hit in res[0]:
        idx = hit.id
        score = float(hit.score)
        meta = {
            "doc_id": hit.entity.get("doc_id"),
            "chunk_id": hit.entity.get("chunk_id")
        }
        out.append((idx, score, texts[idx], meta))

    return out, (t1 - t0)


In [33]:
res5, t5 = milvus_search(query_embedding, k=5, ef=128)   # más preciso
res20, t20 = milvus_search(query_embedding, k=20, ef=32) # más rápido

print("k=5 tiempo:", t5)
print("k=20 tiempo:", t20)

ids_precise = [r[0] for r in res5]
ids_fast = [r[0] for r in res20[:5]]

overlap = len(set(ids_precise).intersection(set(ids_fast)))
print("Overlap (top5 vs top5 fast):", overlap, "/5")


k=5 tiempo: 0.34229493141174316
k=20 tiempo: 0.18275785446166992
Overlap (top5 vs top5 fast): 5 /5


In [34]:
# comparamos top20 con ef alto vs ef bajo
res20_prec, t20_prec = milvus_search(query_embedding, k=20, ef=256)
res20_fast, t20_fast = milvus_search(query_embedding, k=20, ef=8)

ids_prec20 = [r[0] for r in res20_prec]
ids_fast20 = [r[0] for r in res20_fast]

overlap20 = len(set(ids_prec20).intersection(set(ids_fast20)))
print("Tiempo ef=256:", t20_prec, "| Tiempo ef=8:", t20_fast)
print("Overlap top20:", overlap20, "/20")
print("IDs solo en precise:", list(set(ids_prec20) - set(ids_fast20))[:10])
print("IDs solo en fast:", list(set(ids_fast20) - set(ids_prec20))[:10])


Tiempo ef=256: 0.1689591407775879 | Tiempo ef=8: 0.17020201683044434
Overlap top20: 20 /20
IDs solo en precise: []
IDs solo en fast: []


**¿Qué parámetros ajustaste para precisión vs velocidad?**

Ajusté ef (en search) para el trade-off: ef más alto = más precisión y más tiempo; ef más bajo = más rápido.

**¿Qué evidencia tienes de que ANN cambia resultados (aunque sea poco)?**

En esta ejecución particular, el overlap fue 5/5 y 20/20 incluso al reducir significativamente el parámetro ef. Esto indica que, para esta query y este corpus, los vecinos más cercanos son muy estables y fáciles de recuperar. Aun así, el experimento demuestra el mecanismo de control precisión–velocidad mediante ef, y en consultas más ambiguas o datasets más grandes es común observar diferencias en el ranking cuando se reduce este parámetro.

**Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)**

In [35]:
!pip -q install weaviate-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.0 MB/s eta 0:00:00


In [36]:
with open("ri-tarea-3_my_api.txt", "r") as f:
    WEAVIATE_API_KEY = f.read().strip()

WEAVIATE_URL = "https://wpx59s4kro29sqn7m9iznq.c0.us-west3.gcp.weaviate.cloud"


In [37]:
import weaviate
from weaviate.classes.init import Auth
# conectamos a Weaviate cloud
client_w = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)


In [38]:
from weaviate.classes.config import DataType

class_name = "Document"
# recreamos la clase base
if client_w.collections.exists(class_name):
    client_w.collections.delete(class_name)

client_w.collections.create(
    name=class_name,
    vectorizer_config=None,  # usamos embeddings propios
    properties=[
        {"name": "text", "data_type": DataType.TEXT},
        {"name": "doc_id", "data_type": DataType.INT},
        {"name": "chunk_id", "data_type": DataType.INT},
    ],
)


In [39]:
import uuid

collection = client_w.collections.get(class_name)
# cargamos objetos con vectores
with collection.batch.dynamic() as batch:
    for i in range(len(texts)):
        batch.add_object(
            properties={
                "text": texts[i],
                "doc_id": metadatas[i]["doc_id"],
                "chunk_id": metadatas[i]["chunk_id"],
            },
            vector=embeddings[i].tolist(),
            uuid=str(uuid.uuid4()),
        )


In [40]:
def weaviate_search(query_embedding, k=5):
  # consulta near_vector simple
    res = collection.query.near_vector(
        near_vector=query_embedding[0].tolist(),
        limit=k,
        return_metadata=["distance"],
        return_properties=["text", "doc_id", "chunk_id"],
    )

    out = []
    for obj in res.objects:
        out.append((
            obj.uuid,
            obj.metadata.distance,
            obj.properties["text"],
            {
                "doc_id": obj.properties["doc_id"],
                "chunk_id": obj.properties["chunk_id"],
            }
        ))
    return out


In [41]:
results = weaviate_search(query_embedding, k=5)

for r in results:
    print("id:", r[0], "dist:", r[1], "meta:", r[3])
    print("text:", r[2][:200], "...\n")


id: d6712b5e-ec29-4891-89b7-236301ffa2dd dist: 0.12965166568756104 meta: {'doc_id': 1391, 'chunk_id': 0}
text: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it ...

id: 3a9f3d76-7538-4c3b-bfef-20757c8aa21c dist: 0.1381998062133789 meta: {'doc_id': 1, 'chunk_id': 0}
text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It ...

id: da4f2e69-c6e1-4dd3-8501-87339c2423af dist: 0.15989845991134644 meta: {'doc_id': 1391, 'chunk_id': 1}
text: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after having applie ...

id: c0a392e4-d5f7-4492-a808-fd1d187a54d8 dist: 0.1656

**¿Qué diferencia conceptual hay entre “schema + objetos” vs “tabla + filas”?**


En Weaviate, el esquema define clases con propiedades semánticas y objetos que incluyen vectores y metadata, mientras que una tabla SQL almacena filas sin noción directa de similitud semántica ni búsqueda vectorial nativa.


**¿Trade-off de complejidad vs expresividad?**

Weaviate es más expresivo (schema, objetos, búsqueda semántica, filtros integrados), pero requiere aprender su modelo y configuración. SQL es más simple y universal, pero la semántica vectorial se siente “añadida” y menos integrada.


**Parte 6 — Vector Store #4: Chroma (prototipado rápido)**




In [53]:
!pip -q install chromadb

import chromadb

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="wiki_chroma")

batch_size = 5000  # usamos el menor que el máximo permitido

ids_all = [str(i) for i in range(len(texts))]

for i in range(0, len(texts), batch_size):
  # insertamos por bloques grandes
    collection.add(
        ids=ids_all[i:i+batch_size],
        embeddings=embeddings[i:i+batch_size].tolist(),
        documents=texts[i:i+batch_size],
        metadatas=metadatas[i:i+batch_size],
    )

print("Insertado en Chroma:", len(texts))


def chroma_search(query_embedding, k=5):
    q = query_embedding[0].tolist()
  # consultamos top k
    res = collection.query(
        query_embeddings=[q],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    out = []
    for _id, dist, doc, meta in zip(
        res["ids"][0],
        res["distances"][0],
        res["documents"][0],
        res["metadatas"][0]
    ):
        out.append((_id, dist, doc, meta))
    return out

# ejemplo requerido
results = chroma_search(query_embedding, k=5)
for r in results:
    print("id:", r[0], "dist:", r[1], "meta:", r[3])
    print("text:", r[2][:200], "...\n")


Insertado en Chroma: 79104
id: 10176 dist: 0.2593029737472534 meta: {'doc_id': 1391, 'chunk_id': 0}
text: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it ...

id: 1 dist: 0.27639925479888916 meta: {'doc_id': 1, 'chunk_id': 0}
text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It ...

id: 10177 dist: 0.3197968602180481 meta: {'chunk_id': 1, 'doc_id': 1391}
text: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after having applie ...

id: 37406 dist: 0.3217335045337677 meta: {'chunk_id': 1, 'doc_id': 5067}
text: ils. One was connected via a series resistor 

**¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?**


Chroma fue significativamente más simple de implementar, ya que no requiere infraestructura externa ni configuración de esquemas complejos; todo funciona en memoria con pocas líneas de código.


**¿Qué limitaciones ves para un sistema en producción?**


Chroma no está diseñado para alta escalabilidad, concurrencia o despliegues distribuidos, por lo que es más adecuado para prototipos y experimentación que para sistemas productivos a gran escala.

**Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)**

In [43]:
!pip -q install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 119.3 MB/s eta 0:00:00


In [44]:
with open("supabase_key.txt", "r") as f:
    SUPABASE_DB_PASSWORD = f.read().strip()

PG_HOST = "aws-1-us-east-1.pooler.supabase.com"
PG_PORT = 5432
PG_DB   = "postgres"
PG_USER = "postgres.psqtfsnbgvfqyzrphpzi"
PG_PASS = SUPABASE_DB_PASSWORD


In [45]:
import psycopg2

conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    dbname=PG_DB,
    user=PG_USER,
    password=PG_PASS,
    sslmode="require"
)

cur = conn.cursor()
print("Conectado a PostgreSQL (Supabase Session Pooler)")


Conectado a PostgreSQL (Supabase Session Pooler)


In [46]:
# habilitamos extensión pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
print("pgvector habilitado")


pgvector habilitado


In [47]:
# creamos tabla de documentos
cur.execute("""
DROP TABLE IF EXISTS documents;
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    text TEXT,
    doc_id INT,
    chunk_id INT,
    embedding VECTOR(768)
);
""")
conn.commit()


In [48]:
# se limita a 5000 filas para no demorarse horas en la ejecuciòn
MAX_ROWS = 5000
from tqdm import tqdm

insert_sql = """
INSERT INTO documents (text, doc_id, chunk_id, embedding)
VALUES (%s, %s, %s, %s)
"""

batch_size = 500
N = min(len(texts), MAX_ROWS)

for i in tqdm(range(0, N, batch_size)):
    batch = []
    for j in range(i, min(i + batch_size, N)):
        batch.append((
            texts[j],
            metadatas[j]["doc_id"],
            metadatas[j]["chunk_id"],
            embeddings[j].tolist()
        ))
    cur.executemany(insert_sql, batch)
    conn.commit()

print("Insertados:", N)


100%|██████████| 10/10 [19:02<00:00, 114.27s/it]

Insertados: 5000


In [49]:
conn.rollback()


In [50]:
def pgvector_search(query_embedding, k=5):
  # ordenamos por distancia
    cur.execute("""
        SELECT id, text, doc_id, chunk_id,
               embedding <-> %s::vector AS distance
        FROM documents
        ORDER BY embedding <-> %s::vector
        LIMIT %s;
    """, (query_embedding[0].tolist(), query_embedding[0].tolist(), k))

    rows = cur.fetchall()
    out = []
    for r in rows:
        out.append((
            r[0],          # id
            float(r[4]),   # distance
            r[1],          # text
            {"doc_id": r[2], "chunk_id": r[3]}
        ))
    return out


In [51]:
results = pgvector_search(query_embedding, k=5)

for r in results:
    print("id:", r[0], "dist:", r[1], "meta:", r[3])
    print("text:", r[2][:200], "...\n")


id: 2 dist: 0.525736868403662 meta: {'doc_id': 1, 'chunk_id': 0}
text: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of charge. It ...

id: 6 dist: 0.580306781952213 meta: {'doc_id': 1, 'chunk_id': 4}
text: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give a reading of internal resistance for a rechargeabl ...

id: 3 dist: 0.596317287343088 meta: {'doc_id': 1, 'chunk_id': 1}
text: ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether the battery is being charged or discharged. In the ...

id: 15 dist: 0.604379153429689 meta: {'doc_id': 5, 'chunk_id': 0}
text: Capacity loss Capacity loss or capacity fading is a phenomenon observed in rechargeable battery u

**¿Qué tan “explicable” te parece esta aproximación vs las otras?**

Es la más “explicable”, porque se ve directamente en SQL qué se está haciendo: guardar un vector y ordenar por distancia. No es una “caja negra” de motor vectorial.

**¿Ventajas del mundo SQL (JOIN, filtros, agregaciones)?**

Puedes combinar similitud vectorial con filtros tradicionales, JOINs con otras tablas, agregaciones y reglas de negocio en una sola consulta/pipeline.


**¿Limitaciones esperas en escalabilidad vs bases vectoriales dedicadas?**

A gran escala, PostgreSQL puede ser más lento y más difícil de tunear para búsquedas masivas/altamente concurrentes que motores dedicados (Milvus/Qdrant/Weaviate), que están diseñados específicamente para índices ANN, sharding y alto QPS.
